<a href="https://colab.research.google.com/github/chaiminwoo0223/Machine-Learning/blob/main/10_CrossValidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data,target,test_size=0.2,random_state=42)

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier # 결정 트리
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input,sub_target)
print(dt.score(sub_input,sub_target))
print(dt.score(val_input,val_target))

0.9971133028626413
0.864423076923077


In [7]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt,train_input,train_target)
print(scores)

{'fit_time': array([0.0093286 , 0.00724626, 0.00747919, 0.00744152, 0.00854421]), 'score_time': array([0.00086451, 0.00080204, 0.00071597, 0.00068498, 0.00081992]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [8]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [9]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt,train_input,train_target,cv=StratifiedKFold())

In [10]:
splitter = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
scores = cross_validate(dt,train_input,train_target,cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [11]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001,0.0002,0.0003,0.0004,0.0005]}

In [12]:
dt = DecisionTreeClassifier(random_state=42)
gs = GridSearchCV(dt,params,n_jobs=-1)

In [13]:
gs.fit(train_input,train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [14]:
dt = gs.best_estimator_
print(dt.score(train_input,train_target))

0.9615162593804117


In [15]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [16]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [17]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [18]:
params = {'min_impurity_decrease': np.arange(0.0001,0.001,0.0001),
          'max_depth': range(5,20,1),
          'min_samples_split': range(2,100,10)}

In [19]:
gs = GridSearchCV(dt,params,n_jobs=-1)
gs.fit(train_input,train_target)

GridSearchCV(estimator=DecisionTreeClassifier(min_impurity_decrease=0.0001,
                                              random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [20]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [21]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [22]:
from scipy.stats import uniform, randint

In [23]:
rgen = randint(0,10)
rgen.rvs(10)

array([0, 7, 1, 9, 5, 9, 9, 3, 7, 8])

In [24]:
np.unique(rgen.rvs(1000),return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([100,  92, 100,  85, 104,  99, 107, 100, 105, 108]))

In [25]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.74372648, 0.2312975 , 0.06257001, 0.56820749, 0.50919701,
       0.03133343, 0.47084788, 0.26874285, 0.22260794, 0.026531  ])

In [26]:
params = {'min_impurity_decrease': uniform(0.0001,0.001),
          'max_depth': randint(20,50),
          'min_samples_split': randint(2,25),
          'min_samples_leaf': randint(1,25)}

In [27]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(dt,params,n_iter=100,n_jobs=-1,random_state=42)
gs.fit(train_input,train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(min_impurity_decrease=0.0001,
                                                    random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fdbc2684410>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fdbc2684cd0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fdbc26dd990>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fdbc2684950>},
                   random_state=42)

In [28]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [29]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [30]:
best = gs.best_estimator_
print(best.score(test_input,test_target))

0.86
